# 1.0-initial-data-exporation

In [37]:
# Import required libraries
import re
import torch
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertForMaskedLM, BertTokenizerFast

## Review and processing of source data

Firstly, load our dataset and look on it.

In [16]:
df = pd.read_csv('../data/raw/filtered.tsv', sep='\t')
df = df.drop(['Unnamed: 0'], axis=1)

In [17]:
dataset = df.copy()
dataset = dataset.head(20000)
dataset

,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348
...,...,...,...,...,...,...
19995,Now what else did the prostitute say?,what else did the bride say?,0.725349,0.236842,0.978869,0.000040
19996,I'm gonna spew.,I'll puke.,0.699393,0.312500,0.088549,0.978189
19997,"""He is that,"" said Estraven, ""but I asked too ...","""he's a traitor,"" Estraven told me, ""but I ask...",0.693783,0.061224,0.000046,0.964762
19998,"Hello, new dice smell.","hey, the smell of new tits.",0.647981,0.178571,0.000183,0.993284


In [18]:
ref_mean = np.mean(dataset["ref_tox"])
trn_mean = np.mean(dataset["trn_tox"])
print(ref_mean, trn_mean)

0.5416104048595838 0.43504418545919943


We see that toxic sentences located randomly, so we will make a test(little)/final(big) datasets with all toxic sentences in reference column.

In [21]:
for i in dataset.index:
    if df.loc[i, 'trn_tox'] >= 0.5:
        print(i)
        # Swap 'reference' and 'translation'
        df.loc[i, 'reference'], df.loc[i, 'translation'] = df.loc[i, 'translation'], df.loc[i, 'reference']
        
        # Swap 'ref_tox' and 'trn_tox'
        df.loc[i, 'ref_tox'], df.loc[i, 'trn_tox'] = df.loc[i, 'trn_tox'], df.loc[i, 'ref_tox']
df.to_csv('../data/interim/processed_dataset.csv', index=False)

In [23]:
for i in dataset.index:
    if dataset.loc[i, 'trn_tox'] >= 0.5:
        # Swap 'reference' and 'translation'
        dataset.loc[i, 'reference'], dataset.loc[i, 'translation'] = dataset.loc[i, 'translation'], dataset.loc[i, 'reference']
        
        # Swap 'ref_tox' and 'trn_tox'
        dataset.loc[i, 'ref_tox'], dataset.loc[i, 'trn_tox'] = dataset.loc[i, 'trn_tox'], dataset.loc[i, 'ref_tox']
dataset

,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,"if Alkar floods her with her mental waste, it ...","If Alkar is flooding her with psychic waste, t...",0.785171,0.010309,0.981983,0.014195
1,you're becoming disgusting.,Now you're getting nasty.,0.749687,0.071429,0.999039,0.065473
2,"well, we can spare your life.","Well, we could spare your life, for one.",0.919051,0.268293,0.985068,0.213313
3,"monkey, you have to wake up.","Ah! Monkey, you've got to snap out of it.",0.664333,0.309524,0.994215,0.053362
4,I have orders to kill her.,I've got orders to put her down.,0.726639,0.181818,0.999348,0.009402
...,...,...,...,...,...,...
19995,Now what else did the prostitute say?,what else did the bride say?,0.725349,0.236842,0.978869,0.000040
19996,I'll puke.,I'm gonna spew.,0.699393,0.312500,0.978189,0.088549
19997,"""he's a traitor,"" Estraven told me, ""but I ask...","""He is that,"" said Estraven, ""but I asked too ...",0.693783,0.061224,0.964762,0.000046
19998,"hey, the smell of new tits.","Hello, new dice smell.",0.647981,0.178571,0.993284,0.000183


# Logistic regression for word toxicities scores

In [25]:
# Make toxic and nontoxic sentences lists with labels list(1 if sentence is toxic, 0 otherwise)
toxic_sentence = list(dataset['reference'][:20000])
nontoxic_sentence = list(dataset['translation'][:20000])
toxic_labels = [1 for i in range(len(toxic_sentence))]
nontoxic_labels = [0 for i in range(len(nontoxic_sentence))]

In [28]:
# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Prepare the sentences and labels
sentences = toxic_sentence + nontoxic_sentence
labels = toxic_labels + nontoxic_labels

# Vectorize the sentences
X = vectorizer.fit_transform(sentences)

# Train the logistic regression model
lr_model = LogisticRegression()
lr_model.fit(X, labels)

# Get the feature names (words) and their weights from the logistic regression model
feature_names = vectorizer.get_feature_names_out()
feature_weights = lr_model.coef_[0]

# Create a dictionary mapping words to their toxicity scores (weights)
word_toxicity_scores = dict(zip(feature_names, feature_weights))
word_toxicity_scores 

C:\Users\markz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'00': -1.0947997711128292,
 '000': -0.20266760958065058,
 '02': -0.029276249682182936,
 '05': -0.02330705106659507,
 '0ne': -0.01600523820550363,
 '0r': 0.3972637704716108,
 '0um': -0.1169852304765331,
 '10': 0.26850342559177315,
 '100': -0.13840840625929415,
 '1000': -0.5390077794883651,
 '1027': -0.15034745888087384,
 '106': 0.02180603286969878,
 '107': -0.2294174921217973,
 '11': -0.39851487470705454,
 '110': -0.22603492285183516,
 '111': -0.426068633154974,
 '12': -0.0829969122128543,
 '125': -0.06111256309104965,
 '13': -0.10696217167319082,
 '14': -0.39306085159055604,
 '14th': 0.3321202409169549,
 '15': 0.07213021392461777,
 '150': -0.06077184322550835,
 '157': 0.029688444375601628,
 '16': -0.920719230158248,
 '160': 0.45253177702626945,
 '16th': -0.33008914530299854,
 '17': 0.3462202768955719,
 '1785': -0.07315921421635664,
 '18': -0.056167943543308924,
 '182': 0.018869711854004784,
 '1820': 0.13445406326374737,
 '1864': -0.1400772209158153,
 '18th': -0.02876552138328062,
 '19

## Initializing toxic words according to their toxicity scores

In [31]:
# Initialize a list to store toxic words
toxic_words = []

# Iterate over each sentence
for sentence in toxic_sentence:
    # Split the sentence into words, considering punctuation
    words = re.findall(r'\b\w+\b', sentence)
    
    # Compute the toxicity scores for the words
    scores = [word_toxicity_scores.get(word, 0) for word in words]
    
    # Compute the threshold based on the maximum score in the sentence
    threshold = max(0.2, max(scores)/2) if scores else 0.2
    
    # Find the toxic words and add them to the list
    toxic_words.extend(word for word, score in zip(words, scores) if score > threshold)

# Remove duplicates by converting the list to a set and then back to a list
toxic_words = list(set(toxic_words))

toxic_words

['spit',
 'vagrants',
 'wetness',
 'forehead',
 'skulking',
 'shithole',
 'ridiculousness',
 'drowned',
 'supposedto',
 'throat',
 'ture',
 'arrogant',
 'slut',
 'shitless',
 'ambition',
 'whorehouse',
 'disgusting',
 'lout',
 'capsid',
 'lowers',
 'decapitate',
 'bullshit',
 'annihilate',
 'skank',
 'sucked',
 'secret',
 'laziest',
 'haggle',
 'qualify',
 'butterflies',
 'poison',
 'galloper',
 'biting',
 'earthworm',
 'bridges',
 'necking',
 'uttering',
 'radiation',
 'clitoris',
 'stupidest',
 'vagabond',
 'homosexual',
 'plans',
 'whatsoever',
 'drat',
 'pose',
 'bitch',
 'knock',
 'goon',
 'detonates',
 'tarantula',
 'dodgy',
 'flay',
 'bigot',
 'monster',
 'slaughtered',
 'parasite',
 'daybecome',
 'bastards',
 'crazies',
 'dissect',
 'imports',
 'names',
 'pisshole',
 'dumber',
 'carpet',
 'gravely',
 'homophobe',
 'prefer',
 'nerveless',
 'repent',
 'immense',
 'store',
 'arns',
 'grab',
 'grok',
 'puny',
 'boxtroll',
 'maintain',
 'cuff',
 'cuffs',
 'silliness',
 'vaginal',
 '

In [34]:
toxic_sentence[20:30]

['the foolish Xerxes calmly passed all control of the computer network and commanded her to handle any kind of trouble.',
 "That night, Li'l Dice satisfied his thirst to kill, though he knew Shaggy would never forgive him.",
 'Real life starts the first time you fuck, kid.',
 "I think you're the weirdest person I've ever met.",
 "I mean, I think it's fucking crazy I'm talking to you.",
 "Shit, this one I can't even pronounce.",
 'I like that shit.',
 "Trying to keep me fucking drugged so I don't know what's going on.",
 'How is this not porn? This is porn that comes home.',
 'Hey, leave the poor bastard alone!']

## Masking toxic words in sentences

In [41]:
test_text = toxic_sentence[20:30]

for i in range(len(test_text)):
    # Tokenize the sentence into words and punctuation
    tokens = re.findall(r"\b\w+\b|[\.,!?']", test_text[i])
    
    # Replace toxic words with [MASK]
    for j in range(len(tokens)):
        if tokens[j] in toxic_words:
            tokens[j] = '[MASK]'
            
    print("Original sentence:", test_text[i])
    
    # Join the tokens back into a sentence
    test_text[i] = ' '.join(tokens)
    print("Masked sentence:", test_text[i])

Original sentence: the foolish Xerxes calmly passed all control of the computer network and commanded her to handle any kind of trouble.
Masked sentence: the [MASK] Xerxes calmly passed all control of the computer network and commanded [MASK] to handle any kind of trouble .
Original sentence: That night, Li'l Dice satisfied his thirst to kill, though he knew Shaggy would never forgive him.
Masked sentence: That night , Li ' l Dice satisfied [MASK] thirst to [MASK] , though he knew Shaggy would never forgive [MASK] .
Original sentence: Real life starts the first time you fuck, kid.
Masked sentence: Real life starts the first time [MASK] [MASK] , [MASK] .
Original sentence: I think you're the weirdest person I've ever met.
Masked sentence: I think [MASK] ' [MASK] the [MASK] person I ' ve ever met .
Original sentence: I mean, I think it's fucking crazy I'm talking to you.
Masked sentence: I mean , I think it ' s [MASK] [MASK] I ' m talking to [MASK] .
Original sentence: Shit, this one I c

## Simple BERT model for predicting masked tokens in sentences

In [44]:
# Load pre-trained model and tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

for i in range(len(test_text)):
    # Tokenize the sentence
    inputs = tokenizer(test_text[i], return_tensors='pt', truncation=True, padding=True)

    # Predict the masked tokens
    outputs = model(**inputs)
    predictions = outputs.logits

    # Get the predicted tokens
    predicted_ids = torch.argmax(predictions, dim=2)

    # Decode the tokens to strings
    predicted_sentence = tokenizer.decode(predicted_ids[0])

    print("Predicted sentence:", predicted_sentence)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Predicted sentence: . the elder xerxes calmly passed all control of the computer network and commanded it to handle any kind of trouble..
Predicted sentence: . that night, li'l dice took his thirst to death, though he knew she would never forgive him. he
Predicted sentence: . real life starts the first time in time, too..
Predicted sentence: . i think she'm the best person i've ever met..
Predicted sentence: . i mean, i think it's weird because i. m talking to you..
Predicted sentence: .., this one i can. t even ino say..
Predicted sentence: . i like that one..
Predicted sentence: . trying to keep me completely drugged so i don't know what's going on..
Predicted sentence: . how is this not home? this is something that comes home..
Predicted sentence: . hey, leave the poor girl alone!.


While the basic BERT model can predict masked words, its performance appears to be somewhat random and not as accurate as desired. Therefore, for a more effective solution, it would be advisable to either develop a more sophisticated model or utilize a high-performing model available in open source repositories.